#Install

In [1]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
!pip install Shapely

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (2,857 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

In [2]:
from shapely.geometry import Point,Polygon
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
import folium
import plotly_express as px
from google.colab import drive
from IPython.display import display
drive.mount('/content/drive')

Mounted at /content/drive


#Read

In [17]:
district_path =  "/content/drive/My Drive/2 Boundary/District_NE_new.geojson"
roadDB_path = "/content/drive/My Drive/2021 Route Prediction/Project-0/Road Data/roaddb.csv"

In [22]:
district_NE = gpd.read_file(district_path)
roadDB = pd.read_csv(roadDB_path)

#Simple data

In [34]:
district_NE.head()

,AMP_CODE,PRV_CODE,AMP_NAME_T,AMP_NAME_E,Shape_Leng,Shape_Area,geometry
0,3001,30,เมืองนครราชสีมา,Mueang Nakhon Ratchasima,160600.306403,8.856562e+08,"POLYGON ((102.18341 15.13798, 102.18298 15.137..."
1,3002,30,ครบุรี,Khon Buri,261136.836300,2.168627e+09,"POLYGON ((102.15081 14.61461, 102.15115 14.614..."
2,3003,30,เสิงสาง,Soeng Sang,181829.710406,1.097496e+09,"POLYGON ((102.49675 14.50541, 102.49686 14.505..."
3,3004,30,คง,Khong,208115.123225,7.534105e+08,"POLYGON ((102.27233 15.53663, 102.27538 15.536..."
4,3005,30,บ้านเหลื่อม,Ban Lueam,88918.433064,2.569881e+08,"POLYGON ((102.12032 15.64497, 102.12072 15.644..."


In [35]:
district_NE.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   AMP_CODE    321 non-null    object  
 1   PRV_CODE    321 non-null    object  
 2   AMP_NAME_T  321 non-null    object  
 3   AMP_NAME_E  321 non-null    object  
 4   Shape_Leng  321 non-null    float64 
 5   Shape_Area  321 non-null    float64 
 6   geometry    321 non-null    geometry
dtypes: float64(2), geometry(1), object(4)
memory usage: 17.7+ KB


In [36]:
roadDB.head()

,rid,rd,km,ptype,lat,lon,latx,lonx,angle
0,1,3003,0.0,100m,14.492856,100.066108,14.493,100.066,180
1,1,3003,0.1,100m,14.493738,100.066290,14.494,100.066,180
2,1,3003,0.2,100m,14.494619,100.066476,14.495,100.066,179
3,1,3003,0.3,100m,14.495501,100.066658,14.496,100.067,179
4,1,3003,0.4,100m,14.496383,100.066839,14.496,100.067,179


In [37]:
roadDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093655 entries, 0 to 1093654
Data columns (total 9 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   rid     1093655 non-null  int64  
 1   rd      1093655 non-null  int64  
 2   km      1093655 non-null  float64
 3   ptype   1093655 non-null  object 
 4   lat     1093655 non-null  float64
 5   lon     1093655 non-null  float64
 6   latx    1093655 non-null  float64
 7   lonx    1093655 non-null  float64
 8   angle   1093655 non-null  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 75.1+ MB


#Algorithm

In [121]:
def chack_district(lon,lat):
  point = Point(lon,lat)
  for index in district_NE.index:
    if point.within(district_NE.loc[index,'geometry']):
      return district_NE.loc[index,'AMP_NAME_T']
  return 'false'

In [112]:
chack_district(102.433193,17.217769)

'เมืองหนองบัวลำภู'

#Road 1,2 digit

In [176]:
roadDB_digit = roadDB.loc[roadDB['rd']/10 < 10]

In [177]:
roadDB_digit.ptype.unique()

array(['100m', '1km', 'bend', '100m, bend', '1km, bend'], dtype=object)

In [178]:
roadDB_digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle
542625,3379,32,0.000,100m,14.170919,100.615903,14.171,100.616,180
542626,3379,32,0.100,100m,14.171708,100.616347,14.172,100.616,168
542627,3379,32,0.200,100m,14.172564,100.616619,14.173,100.617,159
542628,3379,32,0.300,100m,14.173457,100.616575,14.173,100.617,168
542629,3379,32,0.400,100m,14.174324,100.616331,14.174,100.616,178
...,...,...,...,...,...,...,...,...,...
1093090,6508,12,54.300,100m,16.689217,101.670215,16.689,101.670,180
1093091,6508,12,54.400,100m,16.688457,101.670716,16.688,101.671,180
1093092,6508,12,54.500,100m,16.687694,101.671213,16.688,101.671,180
1093093,6508,12,54.516,bend,16.687576,101.671292,16.688,101.671,33


#Filter RoadDB

In [179]:
for index in roadDB_digit.index:
  roadDB_digit.loc[index,'district'] = chack_district(roadDB_digit.loc[index,'lon'] , roadDB_digit.loc[index,'lat'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [180]:
roadDB_digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle,district
542625,3379,32,0.000,100m,14.170919,100.615903,14.171,100.616,180,false
542626,3379,32,0.100,100m,14.171708,100.616347,14.172,100.616,168,false
542627,3379,32,0.200,100m,14.172564,100.616619,14.173,100.617,159,false
542628,3379,32,0.300,100m,14.173457,100.616575,14.173,100.617,168,false
542629,3379,32,0.400,100m,14.174324,100.616331,14.174,100.616,178,false
...,...,...,...,...,...,...,...,...,...,...
1093090,6508,12,54.300,100m,16.689217,101.670215,16.689,101.670,180,false
1093091,6508,12,54.400,100m,16.688457,101.670716,16.688,101.671,180,false
1093092,6508,12,54.500,100m,16.687694,101.671213,16.688,101.671,180,false
1093093,6508,12,54.516,bend,16.687576,101.671292,16.688,101.671,33,false


In [181]:
len(roadDB_digit.loc[roadDB_digit['district'] == 'false'])

63441

In [186]:
for index in roadDB_digit.index:
  if roadDB_digit.loc[index,'district'] == 'false':
    roadDB_digit.drop(index,axis=0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [187]:
roadDB_digit

,rid,rd,km,ptype,lat,lon,latx,lonx,angle,district
715238,4489,22,0.0,100m,17.321291,104.416470,17.321,104.416,180,กุสุมาลย์
715239,4489,22,0.1,100m,17.321363,104.417409,17.321,104.417,179,เมืองนครพนม
715240,4489,22,0.2,100m,17.321415,104.418349,17.321,104.418,176,เมืองนครพนม
715241,4489,22,0.3,100m,17.321394,104.419290,17.321,104.419,172,เมืองนครพนม
715242,4489,22,0.4,100m,17.321241,104.420215,17.321,104.420,164,เมืองนครพนม
...,...,...,...,...,...,...,...,...,...,...
1092985,6508,12,43.8,100m,16.718843,101.591987,16.719,101.592,179,คอนสาร
1093003,6508,12,45.6,100m,16.707472,101.603346,16.707,101.603,180,คอนสาร
1093004,6508,12,45.7,100m,16.707015,101.604155,16.707,101.604,178,คอนสาร
1093005,6508,12,45.8,100m,16.706528,101.604944,16.707,101.605,170,คอนสาร
